# Running a simple DFT calculation on PACE Supercomputer Cluster

In this lecture, we will learn about running DFT calculations on PACE. We will first learn about PBS submission scripts that are used to submit jobs on PACE. As a member of Medford group, you will have access to two supercomputer clusters here at Georgia Tech: PACE-Phoenix and PACE-Hive. If you are part of the VIP course, you will have access to PACE-ICE cluster. You shouldn't use "#PBS -A GT-amedford6" if you are on PACE-ICE which is requesting resources from your PI's account. If you are on PACE-Phoenix, you will need to request for resources from your PI's account. This means that you are paying for the jobs you are running therefore, these resources should be used judiciously. An example run.sh script to submit a QE job is given below:

In [ ]:
#!/bin/bash
#PBS -l nodes=1:ppn=8
#PBS -l pmem=3GB
#PBS -l walltime=12:00:00
#PBS -A GT-amedford6 
#PBS -N optimizer
#PBS -o stdout
#PBS -e stderr

cd $PBS_O_WORKDIR

source ~/.bashrc

module load anaconda3 #It is optional to load anaconda. You don't need to load it if it's already been loaded in the environment 

source /storage/coda1/p-amedford6/0/shared/rich_project_chbe-medford/medford-share/envs/espresso-6.7MaX-beef-ipi

python calc.py

To submit a job to the PBS system, you have to use the **qsub** command. We usually call this file run.sh for convenience. The command for submitting the job will be **qsub run.sh**. When you do this, you are essentially handing the file to the PBS queuing system which will scan the file for the blocks containing #PBS that specify the resources you are requesting for the job. Each line in this file starting with #PBS specifies something different. The line with tag **-l** are requesting certain resources (nodes and processors per node). You can request the resources based on requirement of your calculations. In this example script, we are requesting for 1 node and 8 processors per node. The next line (#PBS -l pmem=3GB) specifies the memory you request per processor. Here it is 3 GB. Hence the total memory we request for is 24 GB. For Phoenix cluster, the default queue is *inferno* where the jobs are run. -N is the name you want to give the job, -o and -e are the filenames where the outputs and errors will be printed respectively. You can run *pace-whoami* check the maximum resources available for a particular queue. 

If you are submitting a job to PACE-HIVE, you will need to specify the queue using **#PBS -q**. HIVE has different queues like hive, hive-himem, hive-interact which offer different resources and walltime. After the PBS block, you can source your environment and run the code. The next line with change directory (cd) command with an alias for current working directory (PBS_O_WORKDIR) changes to the correct folder. More information about PBS script and submission of jobs on PACE can be found in this [link](https://pace.gatech.edu/pace-user-documentation). 

## Running SPARC DFT code on PACE

For the purpose of VIP course and training, we will be using SPARC to perform DFT calculations. It is relatively easier to compile SPARC on PACE as compared to QE. In this lesson, we will be setting up our own environments for SPARC, compile the code and learn how to perform DFT calculations using the SPARC python interface. 

### Getting Set Up

Let's start by making a new directory. Login to your PACE account and in your data directory, create a directory called "sparc-run". If you don't find a data directory, you can create on by using the mkdir command.

In [ ]:
cd data
mkdir sparc_run
cd sparc_run

For running SPARC, we will need to compile the SPARC code. For using the python interface for SPARC, we will also need to clone and install the sparc-dft-api package and set the environment variables. The following are the commands for loading the modules and setting up the environment file:

In [ ]:
module purge
module load git
module load intel/19.0.5
module load anaconda3

git clone https://github.com/SPARC-X/SPARC.git
cd SPARC/src/

#go to makefile inside the directory src and set the variables as per instructions in SPARC documentation 
#compiling SPARC code while still in src

make clean; make

#after compiling the code, clone the sparc-dft-api package
git clone https://github.com/SPARC-X/sparc-dft-api.git

#install or clone ase
git clone -b 3.20.1 https://gitlab.com/ase/ase.git

After cloning all the required packages, we can set up the environment file for specifically running SPARC DFT calculations. Open a text editor on terminal (for eg. vim/nano) and copy the following lines into the file. You will need to update the existing paths in the following file based on where your directories and SPARC executable is located. The name of this environment file can be sparc_env.sh.

In [ ]:
#loading the modules

module purge
module load intel/19.0.5
module load anaconda3

export PATH=/storage/home/hpaceice1/[username]/data/sparc_run/SPARC/lib:$PATH
export PYTHONPATH=/storage/home/hpaceice1/[username]/data/sparc_run/ase:$PYTHONPATH
export PATH=/storage/home/hpaceice1/[username]/data/sparc_run/ase/bin:$PATH
export PYTHONPATH=/storage/home/hpaceice1/[username]/data/sparc_run/sparc-dft-api/:$PYTHONPATH
export SPARC_PSP_PATH=/storage/home/hpaceice1/[username]/data/sparc_run/sparc-dft-api/sparc/pseudos/PBE_pseudos

if [[ -z "${PBS_NP}" ]]; then
  export ASE_SPARC_COMMAND="/storage/home/hpaceice1/[username]/data/sparc_run/SPARC/lib/sparc -name PREFIX"
else
  export ASE_SPARC_COMMAND="mpirun -np $PBS_NP /storage/home/hpaceice1/[username]/data/sparc_run/SPARC/lib/sparc -name PREFIX"
fi

What is this doing? It is adding things to the PATH and PYTHONPATH. Linux looks for commands and programs to run by checking through the variable PATH to find the program/command we've asked for. Please note that the user should substitute ssahoo41 with your corresponding username. After setting the environment variables in the environment file, we can source it while running the DFT calculation by specifying it in your PBS script:

In [ ]:
source /storage/home/hpaceice1/ssahoo41/data/sparc_env.sh

### Running a calculation using SPARC python interface

Using ASE to run DFT is just like running EMT like we did while going through the ASE lecture. Let's copy the example script below into a new file named run.py, located in a new directory called sparc_test (located inside the sparc_run directory):

In [ ]:
# import lines
from sparc.sparc_core import SPARC
from ase.build import molecule

# make the atoms
atoms = molecule('H2O')
atoms.cell = [[6,0,0],[0,6,0],[0,0,6]]
atoms.center()

# set up the calculator
calc = SPARC(
             KPOINT_GRID=[1,1,1],
             h = 0.2,
             EXCHANGE_CORRELATION = 'GGA_PBE',
             TOL_SCF=1e-5,
             RELAX_FLAG=1,
             PRINT_FORCES=1,
             PRINT_RELAXOUT=1)

# set the calculator on the atoms and run
atoms.set_calculator(calc)
print(atoms.get_potential_energy())

There are now keyword arguments in the calculator. "KPOINT_GRID" are the density at which we sample the inverse space. "h" is the grid spacing of our mesh basis set. "TOL_SCF" is the convergence criteria for SCF cycle. "RELAX_FLAG" is just telling it we want to perform a structural relaxtion. You will need to specify parameters or keyword arguments prior to running DFT calculations. If you are using LDA or GGA_PBE as exchange correlation functionals, you will need to run convergence tests on KPOINT_GRID and h(mesh spacing). 

Let's run it (don't do this normally on headnode.) Note that it has made lots of output files. "sprc-calc.out" is the main one.

At the top are the settings. Many of these are defaults we did not enter. Next we See Self Consistent Field blocks. These are converging the electron density of the structure. Once converged it gets the energy and forces, then moves the atoms down the potential energy surface.

#### Important Note: 

A calculation with higher number of k points and/or higher value of mesh spacing will be more accurate. Sometimes, you will come across convergence issues while running calculations. In such cases, you can look at your main output file (sprc-calc.out) that saves the energies of each SCF cycle and find out where it breaks. There might be a warning for example:

"WARNING: SCF#1 did not converge to desired accuracy!"

You can play around with input mixing parameters of SPARC in such cases. Some parameters that can be modified if SCF is not converging are:

MIXING_VARIABLE, MIXING_PARAMETER, MIXING_HISTORY, MIXING_PRECOND, CHEB_DEGRESS

More information about the input parameters can be found in the SPARC manual that is available on GitHub. 

### Submitting DFT calculations

Let's copy in our sparc python script and PBS batch file. Modify the PBS file to source the SPARC environment and run the script. The PBE script should have the following lines:

In [ ]:
source /storage/home/hpaceice1/[username]/data/sparc_run/sparc_env.sh
python calc_sparc.py

### What is SPARC doing?

In a broad sense, SPARC is solving the schrodinger equation very approximately. It is doing this through an iterative calculation. You begin with some initial guess of the electron density, then refine that by minimizing the energy. Once you've hit some convergence criteria, you terminate this, then you can calculate the energy and forces.

### Exercise: 

1. Clone the SPARC and sprac-dft-api repository from GitHub and set up an environment for running calculations in SPARC on PACE. 

2. Write a script to build a Platinum bulk crystal and run a single-point DFT calculation in SPARC. 